In [55]:
from pathlib import Path
import subprocess as sb
from subprocess import CalledProcessError
import shutil
from sys import exit, argv
from datetime import datetime
from re import match

import numpy as np

np.show_config()
        

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/tch14/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/tch14/home/anaconda3/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/tch14/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/tch14/home/anaconda3/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/tch14/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/tch14/home/anaconda3/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/tch14/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/tch14/h

In [1]:
import subprocess as sb
qstat = sb.check_output(['qstat'], encoding = 'utf8')  
#qstat = sb.check_output(['qstat'], encoding = 'utf8')   
qstat = [row.split() for row in qstat.split('\n')][2:-1]
' '.join([r[0] if len(r)>0 else r for r in qstat])

'1924299[].pbs 1924300[].pbs 1924301[].pbs 1924302[].pbs 1924303[].pbs 1924304[].pbs 1924305[].pbs 1924306[].pbs 1924307[].pbs 1924308[].pbs 1924309[].pbs 1924310[].pbs 1924311[].pbs 1924312[].pbs 1924313[].pbs 1924314[].pbs 1924315[].pbs 1924316[].pbs 1927077.pbs'

In [27]:
%load_ext autoreload
%autoreload 2
%run ~/FKMC/notebooks/notebook_preamble.py
%matplotlib inline
np.seterr(under = 'ignore')
from time import time
from munch import munchify

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Imported pathlib::Path, os, munch::Munch, pickle, logging, itertools::count, matplotlib::colors
Names are pyplot = plt, matplotlib = mpl, numpy = np


In [42]:
submit_dir = Path('/home/tch14/HPC_data/Tsweep_U5_J5_test')
filename = '35_1.npz'
d = munchify(np.load(submit_dir / 'data' / filename, allow_pickle = True))
logs = d['logs'][()]
logs

array([<FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x1554614a6cd0>,
      dtype=object)

In [43]:
[l.N_sites if l else None for l in logs]

[70, 80, 90, 110, 130, 150, 180, 210, 250, 300]

In [39]:
os.uname()

posix.uname_result(sysname='Linux', nodename='cx1-101-17-3.cx1.hpc.ic.ac.uk', release='4.18.0-193.6.3.el8_2.x86_64', version='#1 SMP Wed Jun 10 11:09:32 UTC 2020', machine='x86_64')

In [34]:
import subprocess as sb
import json
from munch import munchify
from pathlib import Path

stdout = sb.run(
    "qstat -f -F json".split(),
    check=True,
    stdout=sb.PIPE,
).stdout

qstat = munchify(json.loads(stdout))
name = 'Large'
for jobid, job in qstat.Jobs.items():
    if name in job.Job_Name:
        print(f'{jobid} {job.Job_Name} {job.job_state}')


1950671[].pbs Large_IPR_D_0 B
1950672[].pbs Large_IPR_D_1 H
1950673[].pbs Large_IPR_D_2 H
1950674[].pbs Large_IPR_D_3 H
1950675[].pbs Large_IPR_D_4 H
1950676[].pbs Large_IPR_D_5 H
1950677[].pbs Large_IPR_D_6 H
1950678[].pbs Large_IPR_D_7 H
1950679[].pbs Large_IPR_D_8 H
1950680[].pbs Large_IPR_D_9 H
1950681[].pbs Large_IPR_D_10 H
1950682[].pbs Large_IPR_D_11 H
1950683[].pbs Large_IPR_D_12 H
1950684[].pbs Large_IPR_D_13 H
1950685[].pbs Large_IPR_D_14 H
1950686[].pbs Large_IPR_D_15 H
1950687[].pbs Large_IPR_D_16 H
1950688[].pbs Large_IPR_D_17 H
1950689[].pbs Large_IPR_D_18 H
1950690[].pbs Large_IPR_D_19 H


In [98]:
stdout = sb.run(
    ["qstat", 
     "-t", "-f",
     "-F", "json",
     #"1950671[].pbs"
    ],
    check=True,
    stdout=sb.PIPE,
).stdout

def qstat(job_id=None, array=False):
    args =  ["qstat", "-f", "-F", "json"]
    if array: args.append('-t')
    if job_id is not None: args.append(job_id)
    return munchify(json.loads(sb.run(args, check=True, stdout=sb.PIPE).stdout))

def pbsnodes(node):
    args =  ["pbsnodes", "-F", "json", node]
    return munchify(json.loads(sb.run(args, check=True, stdout=sb.PIPE).stdout))
    

#dict_keys(['Job_Name', 'Job_Owner', 'resources_used', 'job_state', 'queue', 'server', 'ctime',
#           'Error_Path', 'exec_host', 'exec_vnode', 'group_list', 'Join_Path', 'Keep_Files', 'Mail_Points', 
#           'mtime', 'Output_Path', 'Priority', 'qtime', 'Rerunable', 'Resource_List', 'stime', 'session_id',
#           'jobdir', 'substate', 'Variable_List', 'comment', 'etime', 'run_count', 'eligible_time', 'array_id',
#           'array_index', 'pset', 'project'])

name = 'Large'
for jobid, job in qstat(array=True).Jobs.items():
    if name in job.Job_Name and job.job_state in "R":
        print(f'{jobid} {job.Job_Name} {job.job_state}')
        logs = Path(job.Output_Path.split(':')[-1])
        jid = jobid.split('.')[0]
        
        print("Runtime: ", job.eligible_time)
        
        node = job.exec_vnode.split(':')[0][1:]
        nodeinfo = munchify(json.loads(sb.run(["pbsnodes", "-F", "json", node], check=True, stdout=sb.PIPE,).stdout)).nodes[node]
        if 'resources_available' in nodeinfo: 
            #print(nodeinfo.resources_available)
            print("CPU model: ", nodeinfo.resources_available.cpumodel)
            print("N CPUs: ", nodeinfo.resources_available.ncpus)
            print("Warranty End: ", nodeinfo.resources_available.warranty_end)
        
        
        err = logs / f"{jid}.ER"
        if err.exists():
            tail = err.open().readlines()[-3:]
            if len(tail) > 0:
                print('\nErrors:')
                print("".join(tail))
        
        print('\nPython Logs:')
        log = logs / f"{jid}.pythonlog"
        if log.exists():
            tail = log.open().readlines()[-3:]
            print("".join(tail))
        
        out = logs / f"{jobid}.OU"
        if out.exists():
            tail = out.open().readlines()[-3:]
            print("".join(tail))

1951833[20].pbs Large_IPR_D_0 R
Runtime:  00:00:13
CPU model:  24
N CPUs:  48
Warranty End:  2021-09-02

Python Logs:
Generating initial state as this is the first run with these params
Diagonalisation benchmark: 0.96 seconds
N = 740: 0% through after 0.00m             rejects: classical = nan% quantum = nan% overall = nan%

1951833[21].pbs Large_IPR_D_0 R
Runtime:  00:00:13
CPU model:  24
N CPUs:  48
Warranty End:  2021-09-02

Python Logs:
Generating initial state as this is the first run with these params
Diagonalisation benchmark: 0.97 seconds
N = 740: 0% through after 0.00m             rejects: classical = nan% quantum = nan% overall = nan%

1951833[22].pbs Large_IPR_D_0 R
Runtime:  00:00:13
CPU model:  24
N CPUs:  48
Warranty End:  2021-09-02

Python Logs:
Generating initial state as this is the first run with these params
Diagonalisation benchmark: 0.89 seconds
N = 740: 0% through after 0.00m             rejects: classical = nan% quantum = nan% overall = nan%

1951833[23].pbs Lar

In [14]:
qstat.Jobs['1950671[].pbs'].job_state

'B'